# Libs

In [2]:
import os
import pathlib
import tensorflow as tf

import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

%matplolib inline

UsageError: Line magic function `%matplolib` not found.


# data

In [7]:
list_of_images = "intersection.jpeg, traffic.jpeg, jungle.jpeg, walkingdog.jpeg, us.jpeg, my_street.jpeg".split(", ")

# get_model

In [4]:
# getting the model from
# https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz'

PATH_TO_MODEL_DIR = tf.keras.utils.get_file(
    fname='ssd_resnet101_v1_fpn_640x640_coco17_tpu-8',
    origin=url,
    untar=True)

PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

# download labels
url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt'

PATH_TO_LABELS = tf.keras.utils.get_file(
    fname='mscoco_label_map.pbtxt',
    origin=url,
    untar=False)

category_index = label_map_util.create_category_index_from_labelmap(
    PATH_TO_LABELS,
    use_display_name=True)

# optionaly you can pring existing categories
#print("Here is a list of existing categories:", category_index)

2023-05-24 14:06:28.214605: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# detectionFucntion

In [17]:
detect_objects("my_street.jpeg")

ValueError: Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor(
[[[[153 150 151 255]
   [163 160 161 255]
   [159 156 157 255]
   ...
   [196 189 194 255]
   [196 189 196 255]
   [196 189 196 255]]

  [[155 152 153 255]
   [163 160 161 255]
   [157 154 155 255]
   ...
   [196 189 195 255]
   [197 190 197 255]
   [196 188 196 255]]

  [[149 146 147 255]
   [154 151 152 255]
   [150 147 147 255]
   ...
   [194 187 194 255]
   [196 189 196 255]
   [195 188 195 255]]

  ...

  [[155 148 150 255]
   [150 144 146 255]
   [142 138 139 255]
   ...
   [ 91  95 102 255]
   [ 94  98 105 255]
   [ 88  92  99 255]]

  [[153 150 152 255]
   [144 141 142 255]
   [153 152 153 255]
   ...
   [ 97 100 107 255]
   [ 93  97 104 255]
   [ 96  99 106 255]]

  [[142 143 145 255]
   [143 143 145 255]
   [151 151 153 255]
   ...
   [ 90  94 101 255]
   [ 89  93  99 255]
   [ 89  93 100 255]]]], shape=(1, 779, 1040, 4), dtype=uint8))
  input_signature: (
    TensorSpec(shape=(1, None, No

In [5]:
def detect_objects(img):
    
    img_dir = "./original_img/"
    img_to_np = np.array(Image.open(img_dir + img))
    img_to_tensor = tf.convert_to_tensor(img_to_np)
    
    input_tensor = img_to_tensor[tf.newaxis, ...]
    
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = img_to_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)

    plt.figure(figsize=(15, 10))
    plt.imshow(image_np_with_detections)
    print(img)
    plt.savefig(f"detected_{img}")

# detect

In [12]:
for img in list_of_images:
    detect_objects(img)

intersection.jpeg
traffic.jpeg
